In [1]:
#Define classes needed for datastructure

class node:
    """ A node in a data tree. Represents a directory """
    def __init__(self, name, ID, parentID):
        self.name = name
        self.ID = ID
        self.parentID = parentID
        self.childrenID = []
        self.files = []
        self.filesizes = []
        self.total_contents_size = 0
        self.total_node_size = 0
        
    def addChild(self, childID):
        self.childrenID.append(childID)
    
    def addFile(self, filesize, file):
        self.files.append(file)
        self.filesizes.append(filesize)


class tree:
    """ 
    Creates a data tree structure out of nodes, based reading the lines in the terminal output.
    Then adds up the contents (file) sizes in each node (directory)
    Then sweeps through the nodes recursively, to get the size of each directory level (includes sub-directories) 
    """
    
    def __init__(self, terminal_output, debug=False):
        self.terminal_out = terminal_output
        self.nodes = [node("/", 0, None)] # Root node, name "/", ID 0, no parents
        self.nodeIDs = {"/" : 0}          # node name:ID lookup
        self.currentNode = 0              # Store what node we are in
        self.nodeSizes = 0                # Size of each node
        
        if not debug:
            # Go through the terminal output to create the datastructure
            self.processLines()
            
            # Add up filesizes in each node
            self.sum_contents()
            
            # Sweep through the nodes recursively adding up the total size of everything below each node
            # Start at the root (node 0)
            print("SUMMING TOTAL NODE SIZES")
            print("------------------------")
            self.nodeSweep(0)
            
            # Calculate the sum of all node sizes below 100000
            self.getAnswer(100000)
        
    def processLines(self):
        """ 
        Goes through each line, performing the commands and creating the data structure
        Calls self.readLine() at each line
        """
        
        print("READING LINES")
        print("----------------------")
        for line in self.terminal_out:
            print("line", line)
            self.readLine(line)
        print("\n")
    
    def readLine(self, line):
        """ Reads an individual line in the terminal output, and processes the command, or stores the file"""
        
        splitLine = line.split()
        
        if splitLine[0] == "$" and splitLine[1] == "cd": 
            # $ is a command. cd is change directory. Ignore ls
            # Perform directory change
            if splitLine[2] == "..":
                # Go to parentID node
                self.currentNode = self.nodes[self.currentNode].parentID
                print("cd up 1 level. New ID =", self.currentNode)
            else:
                # Lookup ID of node we are changing to
                # Look in current directory, unless root ("/")
                newDir = splitLine[2]
                if newDir == "/":
                    newID = 0
                else:
                    for childID in self.nodes[self.currentNode].childrenID:
                        if self.nodes[childID].name == newDir:
                            newID = childID
        
                self.currentNode = newID
                print("new dir =", newDir)
                print("new ID =", newID)
                print("Current ID =", self.currentNode)
                
        elif splitLine[0] == "$" and splitLine[1] == "ls":
            # Do nothing
            print("Do noting, listing contents.")
        
        elif splitLine[0] == "dir":
            #Create child node with parent's ID
            #Add node to tree
            dirName = splitLine[1]
            newNodeID = len(self.nodes) # Will be new element in list, index = current len
            parentID = self.currentNode
            
            child = node(dirName, newNodeID, parentID)
            print("Node created. Node name =", child.name, "Node ID =", child.ID, "Node parent =", child.parentID)
            self.addNode(child)
            
        else:
            #Line is specifying a file and size
            #Store in current node, Split filename and size
            filesize = float(splitLine[0])
            file = splitLine[1]
            self.nodes[self.currentNode].addFile(filesize, file)
            print("Added file. Size = ", self.nodes[self.currentNode].filesizes[-1])
            print("Added file. Name = ", self.nodes[self.currentNode].files[-1])
            
    def addNode(self, newNode):
        """ Adds a new child node (sub directory """
        # Store node and name:ID lookup
        self.nodes.append(newNode)
        self.nodeIDs.update({newNode.name : newNode.ID})

        # Add ID to children of current node
        self.nodes[self.currentNode].addChild(newNode.ID)
        print("Add child to current node -", self.nodes[self.currentNode].ID)
        print("Added child ID -", self.nodes[self.currentNode].childrenID[-1])
        
    def sum_contents(self):
        """ Sums up the files in each directory """
        for node in self.nodes:
            node.total_contents_size = sum(node.filesizes)
            
    def nodeSweep(self, nodeID):
        """ Sweep through each branch of the tree, and add up the node (directory) sizes recursively """
        print("\n")
        print("Node ID =", nodeID)
        current_node = self.nodes[nodeID]
        
        if current_node.childrenID:
            # If the node has children, call this function recursively for each child
            print(f"Node {nodeID} has children - {current_node.childrenID}")
            
            for childID in current_node.childrenID:
                print(f"Swapping to child node ID - {childID}")
                child = self.nodes[childID]
                self.nodeSweep(childID)
                current_node.total_node_size = current_node.total_node_size + child.total_node_size
            
            # After the children have all been added, add the contents (files) at this level too
            current_node.total_node_size = current_node.total_node_size + current_node.total_contents_size
            print(f"Node ID {nodeID} size is now {current_node.total_node_size}")
            
        else:
            # If the node has no children, you have reached a leaf of the tree. Take the size of the contents
            print(f"Node {nodeID} has no children")
            current_node.total_node_size = current_node.total_contents_size
            print(f"Node {nodeID} size = {current_node.total_node_size}")
    
    def getAnswer(self, thresholdSize):
        """ Calculates the sum of all nodes below a threshold size"""
        self.nodeSizes = [node.total_node_size for node in self.nodes]
        applicableNodeSizes = [size for size in self.nodeSizes if size <= thresholdSize]

        print("\n")
        print("Answer =", sum(applicableNodeSizes))
            
            

In [2]:
# Read input data
file = "input.txt"

with open(file) as f:
    # Split terminal output into a list of strings for each line
    terminal_out =f.read().split("\n")

del terminal_out[-1] # Delete empty last line


In [3]:
# PART 1 
data = tree(terminal_out)

READING LINES
----------------------
line $ cd /
new dir = /
new ID = 0
Current ID = 0
line $ ls
Do noting, listing contents.
line 187585 dgflmqwt.srm
Added file. Size =  187585.0
Added file. Name =  dgflmqwt.srm
line dir gnpd
Node created. Node name = gnpd Node ID = 1 Node parent = 0
Add child to current node - 0
Added child ID - 1
line 200058 hbnlqs
Added file. Size =  200058.0
Added file. Name =  hbnlqs
line dir jsv
Node created. Node name = jsv Node ID = 2 Node parent = 0
Add child to current node - 0
Added child ID - 2
line dir mfhzl
Node created. Node name = mfhzl Node ID = 3 Node parent = 0
Add child to current node - 0
Added child ID - 3
line dir nljtr
Node created. Node name = nljtr Node ID = 4 Node parent = 0
Add child to current node - 0
Added child ID - 4
line dir nwzp
Node created. Node name = nwzp Node ID = 5 Node parent = 0
Add child to current node - 0
Added child ID - 5
line 61949 qdswp.wfj
Added file. Size =  61949.0
Added file. Name =  qdswp.wfj
line 21980 rbq.hpj
Ad

In [4]:
# PART 2
total_space = 70000000
update_space = 30000000
used_space = data.nodeSizes[0]

free_space = total_space - used_space
required_space = update_space - free_space

# Find first node size >= required_space
sizes = data.nodeSizes
sizes.sort()
sizes = [size for size in sizes if size >= required_space]

print("Answer 2 =", sizes[0])


Answer 2 = 10475598.0
